# EIM 2022-2023: Agresti, Manzi, Scorrano, Serracca

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install light-the-torch && ltt install torch
!pip install torchio
!pip install SimpleITK
!pip install itk

Import librerie

In [ ]:
import os
import SimpleITK as sitk
import nibabel as nib
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import shutil
import random
import PIL

#------- Installazione Librerie Mancanti #-------
!pip install tensorflow
!python3 -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"

#------- Librerie Skimage -------------
import skimage
import skimage.io as img
import skimage.transform as transform
from skimage.transform import rotate
from skimage.restoration import denoise_tv_chambolle
from skimage import img_as_float, img_as_ubyte
from skimage import exposure

#------- Librerie Monai -------------
import monai
from monai.metrics import DiceMetric
from monai.inferers import SimpleInferer
from monai.transforms import (
    ShiftIntensity,
    AsDiscrete,
    SpatialPad,
    RandZoom,
    DataStatsd,
    HistogramNormalized,
    CenterSpatialCrop,
    AddChanneld,
    Rotated,
    BatchInverseTransform,
    HistogramNormalize,
    RandZoom,
    RandHistogramShiftd,
    Rand2DElasticd,
    CenterSpatialCropd,
    RandSmoothFieldAdjustContrast,
    Compose,
    RandScaleCrop,
    RandSpatialCropd,
    EnsureTyped,
    Activations,
    LoadImaged,
    Affine,
    Resized,
    Spacingd,
    RandAffined,
    RandFlipd,
    EnsureChannelFirst,
    ScaleIntensityRanged,
    Activationsd,
    AsDiscreted,
    FillHoles,
    KeepLargestConnectedComponent,
    DataStats,
    AsChannelFirstd,
    AsDiscreted,
    ToTensord,
    EnsureType,
    CropForegroundd,
    RandAdjustContrastd,
    Orientationd,
    SqueezeDimd,
    RandGaussianNoised,
    RandRotated,
    Zoomd,
    GibbsNoised,
    KSpaceSpikeNoised,
    AdjustContrastd
    )
from monai.data import (
    DataLoader,
    Dataset,
    CacheDataset,
    PILReader,
    IterableDataset,
    decollate_batch)
from monai.utils import set_determinism, first
from monai.config import print_config
from monai.networks.nets import UNet, SegResNet
from monai.networks.layers import Norm
from monai.transforms.spatial.dictionary import RandZoomd

#------- Altre Librerie -------------
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import torch
import PIL
import cv2
import torchio as tio
from scipy import ndimage
from scipy.signal import medfilt2d
from scipy.ndimage import binary_fill_holes
from scipy.spatial import distance
from scipy.spatial.distance import directed_hausdorff
import plotly.express as px
import math
import itk as it
import tensorflow as tf

Estrazione immagini

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/3DMRI Challenge/mm3DMRI.zip"

In [ ]:
#ESTRAZIONE IMMAGINI TRAINING SET

main = '/content'
dataFolder = os.path.join(main, 'DATASET','VOLUMES','TRAIN')
outputFolder = os.path.join(main, 'DATASET','SLICES')
if not os.path.isdir(outputFolder):
      os.mkdir(outputFolder)

volumesList = os.listdir(dataFolder)

for i in tqdm(volumesList):

    subjectFolder = os.path.join(dataFolder, i)
    subjectOuputFolder = os.path.join(outputFolder, i)

    if not os.path.isdir(subjectOuputFolder):
      os.mkdir(subjectOuputFolder)

    if not '.DS_Store' in subjectFolder:

        files = [x for x in os.listdir(subjectFolder)]

        slicesToTake =  range(30,120,5) # Vengono selezionate le slices multiple di 5 dalla 30 alla 119

        for j in files:

            if 'seg' in  j:

                vol_file = os.path.join(subjectFolder,j)

                inputVolume = sitk.ReadImage(vol_file)
                numpyVolume = sitk.GetArrayFromImage(inputVolume).astype(np.uint8)>0

                sliceOutputFolder = os.path.join(subjectOuputFolder,j.split('_')[1].split('.')[0])

                if not os.path.isdir(sliceOutputFolder):
                    os.mkdir(sliceOutputFolder)

                for k in  slicesToTake:

                    filename = os.path.join(sliceOutputFolder, '{:04d}.png'.format(k))
                    tempSlice = numpyVolume[k,:,:].astype(np.uint8)*255
                    slicePIL = PIL.Image.fromarray(tempSlice,mode='L')
                    slicePIL.save(filename)

            else:

                vol_file = os.path.join(subjectFolder,j)

                inputVolume = sitk.ReadImage(vol_file)
                numpyVolume = sitk.GetArrayFromImage(inputVolume)

                numpyVolume = numpyVolume/np.max(numpyVolume)*255 # normalizzazione
                numpyVolume = numpyVolume.astype(np.uint8)

                sliceOutputFolder = os.path.join(subjectOuputFolder,j.split('_')[1].split('.')[0])

                if not os.path.isdir(sliceOutputFolder):
                    os.mkdir(sliceOutputFolder)

                for k in  slicesToTake:

                    filename = os.path.join(sliceOutputFolder, '{:04d}.png'.format(k))
                    tempSlice = numpyVolume[k,:,:]
                    slicePIL = PIL.Image.fromarray(tempSlice,mode='L')
                    slicePIL.save(filename)

In [ ]:
#ESTRAZIONE IMMAGINI VALIDATION

main = '/content/'
dataFolder = os.path.join(main, 'DATASET','VOLUMES','VALIDATION')
outputFolder = os.path.join(main + 'DATASET','SLICES_VALIDATION')
if not os.path.isdir(outputFolder):
      os.mkdir(outputFolder)

volumesList = os.listdir(dataFolder)

for i in tqdm(volumesList):

    subjectFolder = os.path.join(dataFolder, i)
    subjectOuputFolder = os.path.join(outputFolder, i)

    if not os.path.isdir(subjectOuputFolder):
      os.mkdir(subjectOuputFolder)

    if not '.DS_Store' in subjectFolder:

        files = [x for x in os.listdir(subjectFolder)]

        slicesToTake =  range(30,120,3) # Viene selezionata una slice ogni 3 dalla 30 alla 120

        for j in files:

            if 'seg' in  j:

                vol_file = os.path.join(subjectFolder,j)

                inputVolume = sitk.ReadImage(vol_file)
                numpyVolume = sitk.GetArrayFromImage(inputVolume).astype(np.uint8)>0

                sliceOutputFolder = os.path.join(subjectOuputFolder,j.split('_')[1].split('.')[0])

                if not os.path.isdir(sliceOutputFolder):
                    os.mkdir(sliceOutputFolder)

                for k in  slicesToTake:

                    filename = os.path.join(sliceOutputFolder, '{:04d}.png'.format(k))
                    tempSlice = numpyVolume[k,:,:].astype(np.uint8)*255
                    slicePIL = PIL.Image.fromarray(tempSlice,mode='L')
                    slicePIL.save(filename)

            else:

                vol_file = os.path.join(subjectFolder,j)

                inputVolume = sitk.ReadImage(vol_file)
                numpyVolume = sitk.GetArrayFromImage(inputVolume)

                numpyVolume = numpyVolume/np.max(numpyVolume)*255 # normalizzazione
                numpyVolume = numpyVolume.astype(np.uint8)

                sliceOutputFolder = os.path.join(subjectOuputFolder,j.split('_')[1].split('.')[0])

                if not os.path.isdir(sliceOutputFolder):
                    os.mkdir(sliceOutputFolder)

                for k in  slicesToTake:

                    filename = os.path.join(sliceOutputFolder, '{:04d}.png'.format(k))
                    tempSlice = numpyVolume[k,:,:]
                    slicePIL = PIL.Image.fromarray(tempSlice,mode='L')
                    slicePIL.save(filename)


In [ ]:
#ESTRAZIONE IMMAGINI TRAINING_AUGMENTED

main = '/content'
dataFolder = os.path.join(main, 'DATASET','VOLUMES','TRAIN')
outputFolder = os.path.join(main, 'DATASET','SLICES_AUGMENTED')
if not os.path.isdir(outputFolder):
      os.mkdir(outputFolder)

volumesList = os.listdir(dataFolder)

for i in tqdm(volumesList):

    subjectFolder = os.path.join(dataFolder, i)
    subjectOuputFolder = os.path.join(outputFolder, i)

    if not os.path.isdir(subjectOuputFolder):
      os.mkdir(subjectOuputFolder)

    if not '.DS_Store' in subjectFolder:

        files = [x for x in os.listdir(subjectFolder)]

        slicesToTake =  range(37,112,5) # viene selezionata una slice ogni 5 dalla 37 alla 112

        for j in files:

            if 'seg' in  j:

                vol_file = os.path.join(subjectFolder,j)

                inputVolume = sitk.ReadImage(vol_file)
                numpyVolume = sitk.GetArrayFromImage(inputVolume).astype(np.uint8)>0

                sliceOutputFolder = os.path.join(subjectOuputFolder,j.split('_')[1].split('.')[0])

                if not os.path.isdir(sliceOutputFolder):
                    os.mkdir(sliceOutputFolder)

                for k in  slicesToTake:

                    filename = os.path.join(sliceOutputFolder, '{:04d}.png'.format(k))
                    tempSlice = numpyVolume[k,:,:].astype(np.uint8)*255
                    slicePIL = PIL.Image.fromarray(tempSlice,mode='L')
                    slicePIL.save(filename)

            else:

                vol_file = os.path.join(subjectFolder,j)

                inputVolume = sitk.ReadImage(vol_file)
                numpyVolume = sitk.GetArrayFromImage(inputVolume)

                numpyVolume = numpyVolume/np.max(numpyVolume)*255 # normalizzazione
                numpyVolume = numpyVolume.astype(np.uint8)

                sliceOutputFolder = os.path.join(subjectOuputFolder,j.split('_')[1].split('.')[0])

                if not os.path.isdir(sliceOutputFolder):
                    os.mkdir(sliceOutputFolder)

                for k in  slicesToTake:

                    filename = os.path.join(sliceOutputFolder, '{:04d}.png'.format(k))
                    tempSlice = numpyVolume[k,:,:]
                    slicePIL = PIL.Image.fromarray(tempSlice,mode='L')
                    slicePIL.save(filename)

In [ ]:
! rm -r '/content/DATASET/TRAIN_IMG'
! rm -r '/content/DATASET/VAL_IMG'
! rm -r '/content/DATASET/TEST_IMG'
! rm -r '/content/DATASET/TRAIN_AUGM_IMG'

In [ ]:
# creazione path per preparzazione del dataset

current_dir = '/content/DATASET'
testFd_tr = os.path.join(current_dir,'TRAIN_IMG')
testFd_val = os.path.join(current_dir,'VAL_IMG')
testFd_test = os.path.join(current_dir,'TEST_IMG')
testFd_tr_augm = os.path.join(current_dir,'TRAIN_AUGM_IMG')
if not os.path.isdir(testFd_tr):
    os.mkdir(testFd_tr),
    os.mkdir(testFd_val),
    os.mkdir(testFd_test)
    os.mkdir(testFd_tr_augm)


In [ ]:
def kernel_gauss(size, sigma):
  v = np.linspace(-(size-1)/2,(size-1)/2,size)
  x, y = np.meshgrid(v,v)
  h = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
  h = h/h.sum()
  return h

In [ ]:
def normalize8(I):
  mn = I.min()
  mx = I.max()

  mx -= mn

  I = ((I - mn)/mx) * 255
  return I.astype(np.uint8)

In [ ]:
dataset_name = '/content/DATASET'
#estraggo le immagini in T1
SLICES_path = os.path.join(dataset_name,'SLICES')
SLICES_VAL_path = os.path.join(dataset_name,'SLICES_VALIDATION')
SLICE_AUGM_path = os.path.join(dataset_name,'SLICES_AUGMENTED')

images = os.listdir(SLICES_path)
images = sorted(images)
val_images = os.listdir(SLICES_VAL_path)
augm_img = os.listdir(SLICE_AUGM_path)

#SUDDIVISIONE DEL TRAINING DATA E DEL TEST DATA
#la quantità di immagini è pari a:
num_img = len(images)
#ripartisco le immagini in 80% training e 20% Test (Le immagini per il validation sono già fornite)
num_train = int(round(num_img*0.8))
num_test = int(num_img - num_train)
#Estraggo le immagini del training e del test
train_images = images[0:num_train]
test_images = images[num_train:]

elenco = images
beta = 255  # max valore del pixel con codifica uint8
size = 5    # dimensione del kernel del filtro gaussiano
sigma = 0.5 # sigma del filtro gaussiano
h = kernel_gauss(size, sigma) # creazione del kernel del filtro gaussiano
toll = 1e-4 # tolleranza


training_data_t1 = []
# preparazione immagini "RGB" training
for i in train_images:
  path_temp = os.path.join(SLICES_path,i)
  if not os.path.isdir(testFd_tr + '/' + i):
    os.mkdir(testFd_tr + '/' + i),
  for j in os.listdir(path_temp + '/seg'):
    t1 = cv2.imread((os.path.join(SLICES_path, i,'t1',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1_eq = exposure.equalize_adapthist(t1, nbins=255)
    t1_norm = normalize8(t1_eq)

    t1ce = cv2.imread((os.path.join(SLICES_path, i,'t1ce',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1ce_eq = exposure.equalize_adapthist(t1ce, nbins=255)
    t1ce_norm = normalize8(t1ce_eq)

    t2 = cv2.imread((os.path.join(SLICES_path, i,'t2',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t2_eq = exposure.equalize_adapthist(t2, nbins=255)
    t2_norm = normalize8(t2_eq)

    seg = cv2.imread((os.path.join(SLICES_path, i,'seg',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];

    image_to_save = np.stack([t1_norm,t1ce_norm,t2_norm], axis=2)
    image_to_save = np.squeeze(image_to_save)

    pz_path = os.path.join(testFd_tr,i);
    path_img_save = os.path.join(pz_path,'img');
    path_seg_save = os.path.join(pz_path,'seg')
    if not os.path.isdir(pz_path):
      os.mkdir(pz_path)
    if not os.path.isdir(path_img_save):
      os.mkdir(path_img_save)
      os.mkdir(path_seg_save)
    img.imsave(os.path.join(path_img_save,j), image_to_save)
    img.imsave(os.path.join(path_seg_save,j), seg)


# preparazione immagini "RGB" test
for i in test_images:
  path_temp = os.path.join(SLICES_path,i)
  if not os.path.isdir(testFd_test + '/' + i):
    os.mkdir(testFd_test + '/' + i),
  for j in os.listdir(path_temp + '/seg'):
    t1 = cv2.imread((os.path.join(SLICES_path, i,'t1',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1_eq = exposure.equalize_adapthist(t1, nbins=255)
    x1, y1, x2, y2 = 0, 0, 30, 30
    t1_background_1 = np.array(t1_eq[y1:y2, x1:x2])
    std_dev_1 = np.std(t1_background_1)
    x1, y1, x2, y2 = 0, 170, 30, 200
    t1_background_2 = np.array(t1_eq[y1:y2, x1:x2])
    std_dev_2 = np.std(t1_background_2)
    x1, y1, x2, y2 = 170, 0, 200, 30
    t1_background_3 = np.array(t1_eq[y1:y2, x1:x2])
    std_dev_3 = np.std(t1_background_3)
    x1, y1, x2, y2 = 170, 170, 200, 200
    t1_background_4 = np.array(t1_eq[y1:y2, x1:x2])
    std_dev_4 = np.std(t1_background_4)
    std_dev_t1 = np.max(np.array([std_dev_1,std_dev_2,std_dev_3,std_dev_4]))
    if not math.isclose(std_dev_t1, 0, abs_tol=toll):
      print("applicato filtro gaussiano all'immagine t1: paziente->", i, " immagine->", j)
      print("deviazione standard = ", std_dev_t1)
      t1_gauss = ndimage.correlate(t1_eq,h)
      t1_eq = t1_gauss
    t1_gibbs = denoise_tv_chambolle(t1_eq, weight=0.01)
    t1_norm = normalize8(t1_gibbs)

    t1ce = cv2.imread((os.path.join(SLICES_path, i,'t1ce',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1ce_eq = exposure.equalize_adapthist(t1ce, nbins=255)
    x1, y1, x2, y2 = 0, 0, 30, 30
    t1ce_background_1 = np.array(t1ce_eq[y1:y2, x1:x2])
    std_dev_1 = np.std(t1_background_1)
    x1, y1, x2, y2 = 0, 170, 30, 200
    t1ce_background_2 = np.array(t1ce_eq[y1:y2, x1:x2])
    std_dev_2 = np.std(t1_background_2)
    x1, y1, x2, y2 = 170, 0, 200, 30
    t1ce_background_3 = np.array(t1ce_eq[y1:y2, x1:x2])
    std_dev_3 = np.std(t1_background_3)
    x1, y1, x2, y2 = 170, 170, 200, 200
    t1ce_background_4 = np.array(t1ce_eq[y1:y2, x1:x2])
    std_dev_4 = np.std(t1_background_4)
    std_dev_t1ce = np.max(np.array([std_dev_1,std_dev_2,std_dev_3,std_dev_4]))
    if not math.isclose(std_dev_t1ce, 0, abs_tol=toll):
      print("applicato filtro gaussiano all'immagine t1ce: paziente->", i, " immagine->", j)
      print("deviazione standard = ", std_dev_t1ce)
      t1ce_gauss = ndimage.correlate(t1ce_eq,h)
      t1ce_eq = t1ce_gauss
    t1ce_gibbs = denoise_tv_chambolle(t1ce_eq, weight=0.01)
    t1ce_norm = normalize8(t1ce_gibbs)


    t2 = cv2.imread((os.path.join(SLICES_path, i,'t2',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t2_eq = exposure.equalize_adapthist(t2, nbins=255)
    x1, y1, x2, y2 = 0, 0, 30, 30
    t2_background_1 = np.array(t2_eq[y1:y2, x1:x2])
    std_dev_1 = np.std(t1_background_1)
    x1, y1, x2, y2 = 0, 170, 30, 200
    t2_background_2 = np.array(t2_eq[y1:y2, x1:x2])
    std_dev_2 = np.std(t1_background_2)
    x1, y1, x2, y2 = 170, 0, 200, 30
    t2_background_3 = np.array(t2_eq[y1:y2, x1:x2])
    std_dev_3 = np.std(t1_background_3)
    x1, y1, x2, y2 = 170, 170, 200, 200
    t2_background_4 = np.array(t2_eq[y1:y2, x1:x2])
    std_dev_4 = np.std(t1_background_4)
    std_dev_t2 = np.max(np.array([std_dev_1,std_dev_2,std_dev_3,std_dev_4]))
    if not math.isclose(std_dev_t2, 0, abs_tol=toll):
      print("applicato filtro gaussiano all'immagine t2: paziente->", i, " immagine->", j)
      print("deviazione standard = ", std_dev_t2)
      t2_gauss = ndimage.correlate(t2_eq,h)
      t2_eq = t2_gauss
    t2_gibbs = denoise_tv_chambolle(t2_eq, weight=0.01)
    t2_norm = normalize8(t2_gibbs)


    seg = cv2.imread((os.path.join(SLICES_path, i,'seg',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];

    image_to_save = np.stack([t1_norm,t1ce_norm,t2_norm], axis=2)
    image_to_save = np.squeeze(image_to_save)

    pz_path = os.path.join(testFd_test,i);
    path_img_save = os.path.join(pz_path,'img');
    path_seg_save = os.path.join(pz_path,'seg')
    if not os.path.isdir(pz_path):
      os.mkdir(pz_path)
    if not os.path.isdir(path_img_save):
      os.mkdir(path_img_save)
      os.mkdir(path_seg_save)
    img.imsave(os.path.join(path_img_save,j), image_to_save)
    img.imsave(os.path.join(path_seg_save,j), seg)


# preparazione immagini "RGB" validation
testFd_val = os.path.join(current_dir,'VAL_IMG')
for i in os.listdir(SLICES_VAL_path):
  path_temp = os.path.join(SLICES_VAL_path,i)
  if not os.path.isdir(testFd_val + '/' + i):
    os.mkdir(testFd_val + '/' + i),
  for j in os.listdir(path_temp + '/seg'):
    t1 = cv2.imread((os.path.join(SLICES_VAL_path, i,'t1',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1_eq = exposure.equalize_adapthist(t1, nbins=255)
    t1_norm = normalize8(t1_eq)

    t1ce = cv2.imread((os.path.join(SLICES_VAL_path, i,'t1ce',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1ce_eq = exposure.equalize_adapthist(t1ce, nbins=255)
    t1ce_norm = normalize8(t1ce_eq)

    t2 = cv2.imread((os.path.join(SLICES_VAL_path, i,'t2',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t2_eq = exposure.equalize_adapthist(t2, nbins=255)
    t2_norm = normalize8(t2_eq)

    seg = cv2.imread((os.path.join(SLICES_VAL_path, i,'seg',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];

    image_to_save = np.stack([t1_norm,t1ce_norm,t2_norm], axis=2)
    image_to_save = np.squeeze(image_to_save)

    pz_path = os.path.join(testFd_val,i);
    path_img_save = os.path.join(pz_path,'img');
    path_seg_save = os.path.join(pz_path,'seg')
    if not os.path.isdir(pz_path):
      os.mkdir(pz_path)
    if not os.path.isdir(path_img_save):
      os.mkdir(path_img_save)
      os.mkdir(path_seg_save)
    img.imsave(os.path.join(path_img_save,j), image_to_save)
    img.imsave(os.path.join(path_seg_save,j), seg)


# preparazione immagini "RGB" training_augmented (data augmenting)
testFd_augm = os.path.join(current_dir,'TRAIN_AUGM_IMG')
for i in os.listdir(SLICE_AUGM_path):
  path_temp = os.path.join(SLICE_AUGM_path,i)
  if not os.path.isdir(testFd_augm + '/' + i):
    os.mkdir(testFd_augm + '/' + i),
  for j in os.listdir(path_temp + '/seg'):
    t1 = cv2.imread((os.path.join(SLICE_AUGM_path, i,'t1',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1_eq = exposure.equalize_adapthist(t1, nbins=255)
    t1_norm = normalize8(t1_eq)


    t1ce = cv2.imread((os.path.join(SLICE_AUGM_path, i,'t1ce',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t1ce_eq = exposure.equalize_adapthist(t1ce, nbins=255)
    t1ce_norm = normalize8(t1ce_eq)


    t2 = cv2.imread((os.path.join(SLICE_AUGM_path, i,'t2',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];
    t2_eq = exposure.equalize_adapthist(t2, nbins=255)
    t2_norm = normalize8(t2_eq)

    seg = cv2.imread((os.path.join(SLICE_AUGM_path, i,'seg',j)),cv2.IMREAD_GRAYSCALE)[20:220,20:220];

    image_to_save = np.stack([t1_norm,t1ce_norm,t2_norm], axis=2)
    image_to_save = np.squeeze(image_to_save)

    pz_path = os.path.join(testFd_augm,i);
    path_img_save = os.path.join(pz_path,'img');
    path_seg_save = os.path.join(pz_path,'seg')
    if not os.path.isdir(pz_path):
      os.mkdir(pz_path)
    if not os.path.isdir(path_img_save):
      os.mkdir(path_img_save)
      os.mkdir(path_seg_save)
    img.imsave(os.path.join(path_img_save,j), image_to_save)
    img.imsave(os.path.join(path_seg_save,j), seg)


In [ ]:
# Creazione dei path e dei dizionari per le cartelle in cui sono contenuti training set, training set augmented, test set e validation set
train_path = '/content/DATASET/TRAIN_IMG'
training_data = []
for i in train_images:
  path_temp = os.path.join(train_path,i,"seg")
  for j in os.listdir(path_temp):
    tempDict = {'image': os.path.join(train_path,i,"img",j),'segmentation': os.path.join(train_path, i,'seg',j)}
    training_data.append(tempDict)

train_augm_path = '/content/DATASET/TRAIN_AUGM_IMG'
training_augm_data = []
for i in augm_img:
  path_temp = os.path.join(train_augm_path,i,"seg")
  for j in os.listdir(path_temp):
    tempDict = {'image': os.path.join(train_augm_path,i,"img",j),'segmentation': os.path.join(train_augm_path, i,'seg',j)}
    training_augm_data.append(tempDict)

test_path = '/content/DATASET/TEST_IMG'
test_data = []
for i in test_images:
  path_temp = os.path.join(test_path,i,"seg")
  for j in os.listdir(path_temp):
    tempDict = {'image': os.path.join(test_path, i,"img",j),'segmentation': os.path.join(test_path, i,'seg',j)}
    test_data.append(tempDict)

val_path = '/content/DATASET/VAL_IMG'
validation_data = []
for i in os.listdir(SLICES_VAL_path):
  path_temp = os.path.join(val_path,i,"seg")
  for j in os.listdir(path_temp):
    tempDict = {'image': os.path.join(val_path, i,"img",j),'segmentation': os.path.join(val_path, i,'seg',j)}
    validation_data.append(tempDict)

In [ ]:
# Prova di visualizzazione immagine
immagine_prova = img.imread("/content/DATASET/TRAIN_IMG/00001/seg/0045.png")
fig = px.imshow(immagine_prova,color_continuous_scale='gray')
fig.show()
dimension = np.shape(immagine_prova)
print(dimension)

In [ ]:
# Inizializzazione parametri modello
ROI_SIZE = [(200),(200)]  # dimensioni immagine da fornire alla rete
ROI_SIZE_RAND = [100,100] # valore utile per un'operazione di data augmentation successiva
BATCH_SIZE = 72
wks = 0

In [ ]:
#TRAINING + DATA AUGMENTATION: TRASFORMAZIONI, DATASET e DATALOADER
train_transforms_0 = Compose(
       [LoadImaged(keys=["image","segmentation"],image_only=False,reader=PILReader()), # caricamento dell'immagine
        AsChannelFirstd(keys=["image"]),
        AddChanneld(keys=["segmentation"]), # portare l'immagine nel formato channel-first (canali, dimensione 0, dimensione 1)
        Resized(keys=["image", "segmentation"], spatial_size=ROI_SIZE, mode = "area"),
        ScaleIntensityRanged(keys=["image", "segmentation"],a_min=0,a_max=255,b_min=0.0,b_max=1.0,clip=True), # riscala i valori all'interno dell'immagine e della segmentazione dal range 0-255 al range 0-1
        AdjustContrastd(keys = ['image'], gamma = 1.5),
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente),
        ToTensord(keys=["image", "segmentation"])])

#rumore gaussiano + rotazione
train_transforms_1 = Compose(
        [LoadImaged(keys=["image","segmentation"],image_only=False,reader=PILReader(), flag = 'gray'), # caricamento dell'immagine
        AsChannelFirstd(keys=["image"]),
        AddChanneld(keys=["segmentation"]), # portare l'immagine nel formato channel-first (canali, dimensione 0, dimensione 1)
        CenterSpatialCropd(keys=["image","segmentation"], roi_size = ROI_SIZE_RAND),
        Resized(keys=["image", "segmentation"], spatial_size=ROI_SIZE, mode = "area"),
        ScaleIntensityRanged(keys=["image", "segmentation"],a_min=0,a_max=255,b_min=0.0,b_max=1.0,clip=True), # riscala i valori all'interno dell'immagine e della segmentazione dal range 0-255 al range 0-1
        AdjustContrastd(keys = ['image'], gamma = 1.5),
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente)
        RandGaussianNoised(keys="image",mean=0,std=0.1,prob=1),
        RandRotated(keys=["image", "segmentation"],prob=0.1,range_x=10.0,padding_mode="zeros"),
        ToTensord(keys=["image", "segmentation"])])

#rumore di Gibbs + randomflip
train_transforms_2 = Compose(
       [LoadImaged(keys=["image","segmentation"],image_only=False,reader=PILReader(), flag = 'gray'), # caricamento dell'immagine
        AsChannelFirstd(keys=["image"]),
        AddChanneld(keys=["segmentation"]), # portare l'immagine nel formato channel-first (canali, dimensione 0, dimensione 1)
        Resized(keys=["image", "segmentation"], spatial_size=ROI_SIZE, mode = "area"),
        ScaleIntensityRanged(keys=["image", "segmentation"],a_min=0,a_max=255,b_min=0.0,b_max=1.0,clip=True), # riscala i valori all'interno dell'immagine e della segmentazione dal range 0-255 al range 0-1
        AdjustContrastd(keys = ['image'], gamma = 1.5),
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente)
        RandFlipd(keys=["image", "segmentation"],spatial_axis=[0],prob=1),
        GibbsNoised(keys=["image"], alpha=0.8),
        ToTensord(keys=["image", "segmentation"])])

# KSpaceSpikeNoised + zoomd
train_transforms_3 = Compose(
       [LoadImaged(keys=["image","segmentation"],image_only=False,reader=PILReader(), flag = 'gray'), # caricamento dell'immagine
        AsChannelFirstd(keys=["image"]),
        AddChanneld(keys=["segmentation"]), # portare l'immagine nel formato channel-first (canali, dimensione 0, dimensione 1)
        Resized(keys=["image", "segmentation"], spatial_size=ROI_SIZE, mode = "area"),
        ScaleIntensityRanged(keys=["image", "segmentation"],a_min=0,a_max=255,b_min=0.0,b_max=1.0,clip=True), # riscala i valori all'interno dell'immagine e della segmentazione dal range 0-255 al range 0-1
        AdjustContrastd(keys = ['image'], gamma = 1.5),
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente)
        Zoomd(keys=["image","segmentation"],zoom=1.3, mode =['area', 'nearest']),
        KSpaceSpikeNoised(keys=["image"], loc=[100,100], k_intensity=13),
        ToTensord(keys=["image", "segmentation"])])

In [ ]:
# applicazione delle trasformazioni e creazione data loader
train_ds_0 = Dataset(data = training_data , transform = train_transforms_0)
train_ds_1 = Dataset(data = training_augm_data[0:round(len(training_augm_data)*0.33)] , transform = train_transforms_1)
train_ds_2 = Dataset(data = training_augm_data[round(len(training_augm_data)*0.33):round(len(training_augm_data)*0.66)] , transform = train_transforms_2)
train_ds_3 = Dataset(data = training_augm_data[round(len(training_augm_data)*0.66):round(len(training_augm_data))] , transform = train_transforms_3)
ConcatDataSet_t1 = torch.utils.data.ConcatDataset([train_ds_0,train_ds_1,train_ds_2,train_ds_3])
train_loader_t1 = DataLoader(ConcatDataSet_t1, batch_size = BATCH_SIZE, num_workers=wks, pin_memory=True)

In [ ]:
# prova di funzionamento
test_patient = first(train_loader_t1)
a = test_patient['image']
b = a.permute(2,3,0,1)
c = np.squeeze(b)
plt.imshow(c[:,:,15,0])
print(c.shape)
print(a.shape)

In [ ]:
#VALIDATION: TRASFORMAZIONI, DATASET e DATALOADER
#rumore rotazione, randomflip
val_transforms = Compose(
       [LoadImaged(keys=["image","segmentation"],image_only=False,reader=PILReader(), flag = 'gray'), # caricamento dell'immagine
        AsChannelFirstd(keys=["image"]), # portare l'immagine nel formato channel-first (canali, dimensione 0, dimensione 1)
        AddChanneld(keys=["segmentation"]),
        Resized(keys=["image", "segmentation"], spatial_size=ROI_SIZE, mode = "area"),
        ScaleIntensityRanged(keys=["image", "segmentation"],a_min=0,a_max=255,b_min=0.0,b_max=1.0,clip=True), # riscala i valori all'interno dell'immagine e della segmentazione dal range 0-255 al range 0-1
        AdjustContrastd(keys = ['image'], gamma = 1.5),
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente)
        ToTensord(keys=["image", "segmentation"])])

val_ds = Dataset(data = validation_data , transform = val_transforms)
val_loader = DataLoader(val_ds, batch_size = BATCH_SIZE, num_workers=wks, pin_memory=True)

In [ ]:
#prova: Verifica Funzionamento Loader del Training Set -----
test_val_patient = first(val_loader)
a = test_val_patient['image']
b = a.permute(2,3,0,1)
c = np.squeeze(b)
plt.imshow(c[:,:,2,0])
print(c.shape)

In [ ]:
# creazione rete

max_epochs = 70 # numero epoche
# Definiamo il device da utilizzare: GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'''
model = UNet(
    spatial_dims=2,
    in_channels=3,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)
'''

# definizione parametri rete
device = torch.device("cuda:0")
model = SegResNet(
    spatial_dims = 2,           # dimensione input (immagine 2D in questo caso)
    blocks_down=[1, 2, 2, 4],   # numero di blocchi di downsample per ogni layer
    blocks_up=[1, 1, 1],        # numero di sample block per ogni layer
    init_filters=16,            # numero di canali di output per il layer convoluzionale iniziale
    in_channels=3,              # canali in ingresso ("RGB" --> 3)
    out_channels=1,             # canali in uscita (immagine binaria)
    dropout_prob=0.3,           # probabilità per un elemento di essere azzerato
).to(device)

# Definizione del tipo di Inferenza da applicare con il modello allenato:
inferer = SimpleInferer()

# Definizione un seed per renderere ripetibili tutte le opearazioni che andiamo ad effettuare:
set_determinism(seed=46)

# Definizione della loss function
loss_function = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)
torch.backends.cudnn.benchmark = True

# Definizione delle trasformazioni MONAI di post-processing
post_pred = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.6)])
post_label = Compose([EnsureType(), AsDiscrete(threshod=0.5)])

In [ ]:
#CREAZIONE CARTELLA DI DESTINAZIONE DEGLI OUTPUT
current_dir = '/content/drive/MyDrive/Colab Notebooks/DEF_ 3DMRI Challenge'
testFd = os.path.join(current_dir,'OUTPUTS_29_01_23t1_rgb')
if not os.path.isdir(testFd):
    os.mkdir(testFd)

Allenamento rete

In [ ]:
# funzione train che viene richiamata durante l'allenamento della rete

def train(global_step, train_loader, dice_val_best, global_step_best):
    model.train() # setta il modello in modalità training
    epoch_loss = 0
    step = 0

    epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True) # crea un oggetto tqdm per tracciare l'avenzamento delle epoche

    for step, batch in enumerate(epoch_iterator): # ad ogni ciclo for estrae una coppia step (numero dell'iterazione), batch (immagine e segmentazione) dal train loader
        step += 1
        x, y = (batch["image"].cuda(), batch["segmentation"].cuda()) # manda i tensori di immagine e maschera alla GPU

        optimizer.zero_grad() # resetta i gradienti dell'optimizer

        logit_map = model(x).sigmoid() # applica una sgimoide all'output della rete

        loss = loss_function(logit_map, y) # calcolo della BCE loss
        loss.backward() # effettua la backpropagation della loss
        lr_scheduler.step()
        epoch_loss += loss.item() # accumula i valori delle loss nei vari batch
        optimizer.step() # calcola i gradienti e aggiorna l'optimizer

        epoch_iterator.set_description("Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss)) # aggiorna il counter tqdm

        if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations: # condizione per effettuare la validazione

            epoch_iterator_val = tqdm(val_loader, desc="Validate (X Steps) (dice=X.X)", dynamic_ncols=True)
            GB = global_step
            dice_val = validation(epoch_iterator_val,GB) # richiama la funzione di validazione per estrarre la metrica da valutare sul test set
            epoch_loss /= step # divide la loss in base al numero del batch
            epoch_loss_values.append(epoch_loss) # salva i valori della loss
            metric_values.append(dice_val) # salva i valori della metrica

            # salvataggio del modello se la metrica di validazione è migliorata
            if dice_val > dice_val_best:
                dice_val_best = dice_val
                global_step_best = global_step
                torch.save( model.state_dict(), os.path.join(current_dir, "best_metric_model.pth"))
                print("Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val))
            else:
                print("Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val))

        global_step += 1

    return global_step, dice_val_best, global_step_best

In [ ]:
# funzione validation che viene richiamata durante l'allenamento

def validation(epoch_iterator_val, GB):
    model.eval() # setta il modello in moadlità inference
    dice_vals = list()
    epoch_val_loss = 0

    with torch.no_grad():
        for step, batch in enumerate(epoch_iterator_val):

            val_inputs, val_labels = (batch["image"].to(device),batch["segmentation"].to(device))


            val_outputs = inferer(val_inputs, model) # applica il modello all'input


            val_inputs = val_inputs.permute(0,3,2,1)

            loss_val = loss_function(val_outputs.sigmoid(), val_labels)
            epoch_val_loss += loss_val.item()

            val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
            val_labels  = [post_label(i) for i in decollate_batch(val_labels)]

            dice_metric(y_pred=val_outputs, y=val_labels)

            dice = dice_metric.aggregate().item()
            dice_vals.append(dice)

            epoch_iterator_val.set_description("Validate (%d Steps) (dice=%2.5f)" % (step, dice))

            fdName = str.split(batch['image_meta_dict']['filename_or_obj'][0],'/')[-1][:-4]
            fd_num = str.split(batch['image_meta_dict']['filename_or_obj'][0],'/')[4]
            fetta = str.split(batch['image_meta_dict']['filename_or_obj'][0],'/')[6]

            stepFd = os.path.join(testFd, str(GB))

            if not os.path.isdir(stepFd):
                os.mkdir(stepFd)

            val_outputs = val_outputs[0][0].detach().cpu().numpy()*255

            dim = ROI_SIZE
            testFd_1 = os.path.join(stepFd,fd_num)
            if not os.path.isdir(testFd_1):
              os.mkdir(testFd_1)


            val_labels = val_labels[0][0].detach().cpu().numpy().squeeze()*255
            val_inputs = val_inputs[0].detach().cpu().numpy().squeeze()*255

            val_inputs = np.flipud(val_inputs.astype(np.uint8))
            val_outputs = rotate(val_outputs,90).astype(np.uint8) # MONAI rotates output and labels
            val_labels = rotate(val_labels,90).astype(np.uint8)

            pr_mask = np.expand_dims(val_outputs, axis=2)
            rgb_muscle_pred = np.concatenate((pr_mask, pr_mask, pr_mask), axis=2)

            gt_mask = np.expand_dims(val_labels, axis=2)
            rgb_muscle_gt = np.concatenate((gt_mask, gt_mask, gt_mask), axis=2)

            stack = np.concatenate((val_inputs, rgb_muscle_gt,rgb_muscle_pred), axis=1).astype(np.uint8)

            stackPIL = PIL.Image.fromarray(stack)
            stackPIL.save(os.path.join(testFd_1,'{}.png'.format(fetta)))
        dice_metric.reset()
        epoch_val_loss /= step + 1
        epoch_val_loss_values.append(epoch_val_loss)

    mean_dice_val = np.mean(dice_vals)

    return mean_dice_val

In [ ]:
data = len(train_ds_0.data) + len(train_ds_1.data) + len(train_ds_2.data) + len(train_ds_3.data)
#viene conteggiata un'epoca quando (batch_size * number_iteration) > = number_data
eval_num = int(data/BATCH_SIZE)+1
max_iterations = int(max_epochs * eval_num)


print(data)
print(max_epochs)
print(max_iterations)
print(eval_num)


dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
global_step = 0
dice_val_best = 0
global_step_best = 0
epoch_loss_values = []
epoch_val_loss_values = []
metric_values = []

In [ ]:
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(global_step, train_loader_t1, dice_val_best, global_step_best)

In [ ]:
# grafici utili: andamento del DICE e della loss in funzione delle epoche
# (nel grafico, nelle ascisse figura il numero di step); andamento della
plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Iteration Average Loss")
x = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
y = epoch_loss_values
y1 = epoch_val_loss_values
plt.xlabel("Iteration")
plt.plot(x, y,label="train BCE")
plt.plot(x,y1,label="val BCE")
plt.legend(loc="upper right")
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [eval_num * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("Iteration")
plt.plot(x, y,color='orange',label="val Dice")
plt.legend(loc="upper left")

plt.show()

Test della rete (su test set)

In [ ]:
current_dir = '/content/drive/MyDrive/Colab Notebooks/3DMRI Challenge'
#definizione rete
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'''
model = UNet(
    spatial_dims=2,
    in_channels=3,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)
'''

device = torch.device("cuda:0")
model = SegResNet(
    spatial_dims = 2,
    blocks_down=[1, 2, 2, 4],
    blocks_up=[1, 1, 1],
    init_filters=16,
    in_channels=3,
    out_channels=1,
    dropout_prob=0.3,
).to(device)

#CARICAMENTO DEL MODELLO
model.load_state_dict(torch.load(current_dir + "SegResNet2.pth"))
inferer = SimpleInferer()

torch.backends.cudnn.benchmark = True

dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)


In [ ]:
# Trasformazioni del test set

test_transforms_0 = Compose(
       [LoadImaged(keys=["image","segmentation"],image_only=False,reader=PILReader(), flag = 'gray'), # caricamento dell'immagine
        AsChannelFirstd(keys=["image"]),
        AddChanneld(keys=["segmentation"]),
        Resized(keys=["image", "segmentation"], spatial_size=ROI_SIZE, mode = "area"),
        ScaleIntensityRanged(keys=["image", "segmentation"],a_min=0,a_max=255,b_min=0.0,b_max=1.0,clip=True), # riscala i valori all'interno dell'immagine e della segmentazione dal range 0-255 al range 0-1
        AdjustContrastd(keys = ['image'], gamma = 1.5),
        AsDiscreted(keys=["segmentation"],threshold=0.5), # porta la segmentazione in formato binario con soglia 0.5 (abbiamo modificato il range di valori nella trasformazione precedente),
        ToTensord(keys=["image", "segmentation"])])

test_ds = IterableDataset(data = test_data, transform = test_transforms_0)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=wks, pin_memory=True)

model.eval()
dice_tests = list()
epoch_test_loss = 0
epoch_test_loss_values = []
min_number_of_pixel = 2000
FP_tot = list()
n_of_pixel = list()
TP_tot = list()
with torch.no_grad():
    for step, batch in enumerate(test_loader):

        test_inputs, test_labels = (batch["image"].to(device),batch["segmentation"].to(device))

        test_outputs = inferer(test_inputs, model)

        test_inputs = test_inputs.permute(0,3,2,1)

        loss_test = loss_function(test_outputs.sigmoid(), test_labels)
        epoch_test_loss += loss_test.item()

        test_outputs = [post_pred(i) for i in decollate_batch(test_outputs)]
        test_labels  = [post_label(i) for i in decollate_batch(test_labels)]

        # compute metric for current iteration
        dice_metric(y_pred=test_outputs, y=test_labels)

        dice = dice_metric.aggregate().item()
        dice_tests.append(dice)

        testFd = os.path.join(current_dir,'TESTING')

        if not os.path.isdir(testFd):
            os.mkdir(testFd)
        maskFd = os.path.join(current_dir,'TESTING','NET_MASK')
        if not os.path.isdir(maskFd):
          os.mkdir(maskFd)

        for jj in range(test_inputs.shape[0]):
          stepFd = os.path.join(testFd, str(jj))
          fdName = str.split(batch['image_meta_dict']['filename_or_obj'][jj],'/')[-1][:-4]
          flag = 0
          test_inputs_slice = test_inputs[jj].detach().cpu().numpy().squeeze()*255
          test_inputs_slice = transform.resize(test_inputs_slice,(240,240))
          test_inputs_slice = np.array(test_inputs_slice, dtype=np.uint8)
          # Estrai i canali R, G, B dell'immagine
          r, g, b = test_inputs_slice[:, :, 0], test_inputs_slice[:, :, 1], test_inputs_slice[:, :, 2]
          non_zero_pixels_r = np.sum(r != 0)
          non_zero_pixels_g = np.sum(g != 0)
          non_zero_pixels_b = np.sum(b != 0)
          non_zero_pixels = np.max(np.array([non_zero_pixels_r,non_zero_pixels_g,non_zero_pixels_b]))
          if non_zero_pixels < min_number_of_pixel:
            flag = 1

          # operazioni di post processing
          test_outputs_slice = test_outputs[jj][0].detach().cpu().numpy().squeeze()*255
          test_outputs_slice = transform.resize(test_outputs_slice,(240,240))
          test_outputs_slice = rotate(test_outputs_slice,90).astype(np.uint8)
          test_outputs_slice = binary_fill_holes(test_outputs_slice)  # riempimento buchi
          test_outputs_slice = np.flipud(test_outputs_slice.astype(np.uint8))
          if flag == 0: # l'oggetto nell'immagine è sufficientemente grande
            test_outputs_slice = np.where(test_outputs_slice==1,255,0)
          if flag == 1: #  l'oggetto nell'immagine è troppo piccolo per una segmentazione corretta
            test_outputs_slice = np.where(test_outputs_slice==1,0,0) # la segmentazione automatica viene posta uguale a 0

          test_outputs_slice = np.array(test_outputs_slice, dtype=np.uint8)
          test_outputs_slice = np.expand_dims(test_outputs_slice, axis=2)
          test_outputs_slice = np.concatenate((test_outputs_slice,test_outputs_slice,test_outputs_slice), axis=2)

          test_labels_slice = test_labels[jj][0].detach().cpu().numpy().squeeze()*255
          test_labels_slice = transform.resize(test_labels_slice,(240,240))
          test_labels_slice = rotate(test_labels_slice,90).astype(np.uint8)
          test_labels_slice = np.flipud(test_labels_slice.astype(np.uint8))
          test_labels_slice = np.array(test_labels_slice, dtype=np.uint8)
          test_labels_slice = np.expand_dims(test_labels_slice, axis=2)
          test_labels_slice = np.concatenate((test_labels_slice,test_labels_slice,test_labels_slice), axis=2)

          FP = len(np.where((test_labels_slice==0)&(test_outputs_slice==255))[0])
          FP_tot.append(FP)
          TP = len(np.where((test_labels_slice==255)&(test_outputs_slice==255))[0])
          TP_tot.append(TP)
          n_of_pixel.append(non_zero_pixels)

          pz_path = os.path.join(testFd,str.split(batch['image_meta_dict']['filename_or_obj'][jj],'/')[4])
          if not os.path.isdir(pz_path):
            os.mkdir(pz_path)
          stack = np.concatenate((test_inputs_slice, test_labels_slice, test_outputs_slice), axis=1).astype(np.uint8)
          stackPIL = PIL.Image.fromarray(stack)
          stackPIL.save(os.path.join(pz_path,'{}.png'.format(fdName)))
          #salvataggio maschere di output in una cartella separatata per il calcolo successivo delle metriche
          maskFd = os.path.join(current_dir,'TESTING','NET_MASK',str.split(batch['image_meta_dict']['filename_or_obj'][jj],'/')[4])
          if not os.path.isdir(maskFd):
            os.mkdir(maskFd)
          maskPIL = PIL.Image.fromarray(test_outputs_slice)
          maskPIL.save(os.path.join(maskFd,'{}.png'.format(fdName)))

    dice_metric.reset()
    epoch_test_loss /= step + 1
    epoch_test_loss_values.append(epoch_test_loss)

mean_dice_test = np.mean(dice_tests)
error_dice_test = np.std(dice_tests)
errore_sulla_media = np.std(dice_tests) / np.sqrt(len(dice_tests))
print("{:.3f} +/- {:.3f}".format(mean_dice_test, error_dice_test))
print("errore sulla media: {:.3f}".format(errore_sulla_media))

In [ ]:
# Grafico per dimostrare l'elevato numero di falsi positivi sotto una certa dimensione
# degli oggetti nelle immagini
vec1 = np.array(n_of_pixel)
vec2 = np.array(FP_tot)
vec3 = np.array(TP_tot)
indici = np.where(vec1<7000)
plt.scatter(vec1[indici[0]], vec2[indici[0]], label= "stars", color= "magenta",
            marker= "*", s=30)
plt.scatter(vec1[indici[0]], vec3[indici[0]], label= "stars", color= "green",
            marker= "*", s=30)
plt.axvline(2000, color='r', linestyle='-')
plt.xlabel("Area Encefalo")
plt.ylabel("Pixel Falsi Positivi")
plt.xlim(0, 7000)
plt.ylim(0, 6000)
plt.show()

In [ ]:
# Ricostruzione dei volumi 3D

main = '/content'
dataFolder_original = os.path.join(main, 'DATASET','VOLUMES','TRAIN')
dataFolder = '/content/drive/MyDrive/Colab Notebooks/DEF_ 3DMRI Challenge/TESTING/NET_MASK'
outputFolder = '/content/drive/MyDrive/Colab Notebooks/DEF_ 3DMRI Challenge/ricostruzione_volumi_output'

if not os.path.isdir(outputFolder):
      os.mkdir(outputFolder) # creazione cartella output

volumesList = os.listdir(dataFolder) # lista dei volumi da ricostruire a partire dalle segmentazioni automatiche presenti in dataFolder

for i in tqdm(volumesList):
    slices = []
    subjectFolder = os.path.join(dataFolder, i)
    subjectOuputFolder = os.path.join(outputFolder, i)

    if not os.path.isdir(subjectOuputFolder):
      os.mkdir(subjectOuputFolder)

    if not '.DS_Store' in subjectFolder:

        files = [x for x in os.listdir(subjectFolder)]

        for j in np.sort(files):
          img = sitk.ReadImage(os.path.join(subjectFolder,j)) #caricamento segmentazioni
          img_vol = sitk.GetArrayFromImage(img).astype(np.uint8)>0
          tempSlice = img_vol[:,:,0].astype(np.uint8)*255
          slices.append(np.array(tempSlice))
          inputVolume = np.stack(slices, axis=-1)

        inputVolume = inputVolume/np.max(inputVolume)*255
        inputVolume = inputVolume.astype(np.uint8)
        numpyVolume_tens = torch.FloatTensor(inputVolume)
        numpyVolume_tens = numpyVolume_tens.permute(2,0,1)
        newVolume = sitk.GetImageFromArray(numpyVolume_tens) # l'array di numeri ottenuto dalla concatenazione delle slice viene trasformato
              # in formato immagine in modo da poter essere comparato con il volume originale che viene letto di seguito nella cartella
              # di origine

        orginal_volume = os.path.join(dataFolder_original,i) # lettura del volume originale per l'estrazione delle informazioni spaziali di ogni slice
        original_volume_list = os.listdir(orginal_volume)[0]
        path_orig_vol = os.path.join(orginal_volume,original_volume_list)
        inputVolume_orig = sitk.ReadImage(path_orig_vol)
        numpyVolume_orig = sitk.GetArrayFromImage(inputVolume_orig).astype(np.uint8)>0

        newVolume.CopyInformation(inputVolume_orig) # le informazioni spaziali vengono trasferite supponendo che le slices siano state nominate durante tutto
            # il processo in egual modo rispetto a come sono state estratte inizialmente
        out = os.path.join(subjectOuputFolder,'volume_output_' + i + '.nii')
        sitk.WriteImage(newVolume, out) # salvataggio in formato nifti (.nii)

In [ ]:
# funzione per il calcolo del Jaccard Index
def jaccard_index(image1, image2):
    set1 = set(np.ravel(image1))
    set2 = set(np.ravel(image2))
    jaccard_index = len(set1 & set2) / len(set1 | set2)
    return jaccard_index

In [ ]:
# funzione per il calcolo della Hausdorff Distance al 95esimo percentile
def HD95(image1, image2):
  nonzero_indices1 = np.nonzero(image1)
  nonzero_indices2 = np.nonzero(image2)
  nonzero_coords1 = np.column_stack(nonzero_indices1)
  nonzero_coords2 = np.column_stack(nonzero_indices2)
  eucl = distance.cdist(nonzero_coords1, nonzero_coords2, 'euclidean')
  minimiA = np.min(eucl, axis=1)
  minimiB = np.min(eucl, axis=0)
  percentile = 95
  resultA = np.percentile(minimiA, percentile, interpolation='nearest') #qui dobbiamo decidere se nearest o default, secondo me è nearest
  resultB = np.percentile(minimiB, percentile, interpolation='nearest')
  Hausdorff_distance_95 = np.max([resultA, resultB])
  return Hausdorff_distance_95

In [ ]:
# sezione per il calcolo delle metriche

pz_dir = os.listdir("/content/drive/MyDrive/Colab Notebooks/3DMRI Challenge/TESTING/NET_MASK")
tot_dice = list()
errore_area = list()
tot_jaccard = list()
tot_hausdorff = list()

for i in pz_dir:
  pz_path_out = os.path.join("/content/drive/MyDrive/Colab Notebooks/3DMRI Challenge/TESTING/NET_MASK",i)
  pz_path_in = os.path.join("/content/DATASET/SLICES",i,"seg")
  segmentation_list = os.listdir(pz_path_out)
  for t in segmentation_list:
    seg_path_out = os.path.join(pz_path_out,t)
    seg_path_in = os.path.join(pz_path_in,t)
    seg_out = cv2.imread(seg_path_out)[:,:,0]
    seg_in = cv2.imread(seg_path_in)[:,:,0]
    # DICE
    TP = len(np.where((seg_in==255)&(seg_out==255))[0])
    FP = len(np.where((seg_in==0)&(seg_out==255))[0])
    FN = len(np.where((seg_in==255)&(seg_out==0))[0])
    if (2*TP+FP+FN) == 0:
      dice = 1
      tot_dice.append(dice)
    else:
      dice = (2*TP)/(2*TP+FP+FN)
      tot_dice.append(dice)
    # DIFFERENZA DI AREE
    seg_out = np.where(seg_out==255,1,0)
    seg_in = np.where(seg_in==255,1,0)
    area_out = np.sum(seg_out)
    area_in = np.sum(seg_in)
    diff_area = area_out - area_in
    errore_area.append(diff_area)
    # JACCARD INDEX
    j_ind = jaccard_index(seg_out,seg_in)
    tot_jaccard.append(j_ind)
    # HAUSDORFF DISTANCE 95%
    nonzero_indices1 = np.nonzero(seg_in)
    nonzero_indices2 = np.nonzero(seg_out)
    if ((len(nonzero_indices1[0])!=0) and (len(nonzero_indices2[0])!=0)):
      hausdorff_distance = HD95(seg_in, seg_out)
      tot_hausdorff.append(hausdorff_distance)


media_dice = np.mean(tot_dice)
errore_dice = np.std([x for x in tot_dice if x != 0 and x != 1])
print("DICE = {:.3f}+/-{:.3f}".format(media_dice, errore_dice))
print("ERRORE AREA MAX = {:.0f}".format(np.max(errore_area)))
print("ERRORE AREA MIN = {:.0f}".format(np.min(errore_area)))
print("ERRORE AREA MEDIO = {:.0f}".format(int(np.mean(errore_area))))
media_jaccard = np.mean(tot_jaccard)
errore_jaccard = np.std(tot_jaccard)
print("JACCARD = {:.3f}+/-{:.3f}".format(media_jaccard, errore_jaccard))
media_hausdorff = np.mean(tot_hausdorff)
errore_hausdorff = np.std(tot_hausdorff)
print("HAUSDORFF = {:.3f}+/-{:.3f}".format(media_hausdorff, errore_hausdorff))